# code created by Claude: 

In [ ]:
# Stock Price Forecasting Pipeline with Flume, HDFS, and PySpark
# Running on Vertex AI Workbench
# This pipeline:
# 1. Ingests historical stock data using Flume
# 2. Stores data in HDFS
# 3. Performs data cleaning and analysis with PySpark
# 4. Builds a multivariate time series forecasting model
# 5. Deploys the model for real-time prediction

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# PySpark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, date_format, to_date, window, lit, expr, when
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline, PipelineModel

# Prophet for time series forecasting
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

# Flask for API deployment
from flask import Flask, request, jsonify

# Airflow for scheduling
from airflow import DAG
from airflow.operators.python_operator import PythonOperator

# Vertex AI imports
from google.cloud import aiplatform
from google.cloud.aiplatform import model_monitoring

###########################################
# 1. FLUME CONFIGURATION Create Flume configuration file for stock data ingestion
###########################################

def create_flume_config():
    
    flume_config = """
    # Stock data source agent
    stockAgent.sources = httpSource
    stockAgent.channels = memoryChannel fileChannel
    stockAgent.sinks = hdfsSink

    # HTTP Source for API data
    stockAgent.sources.httpSource.type = http
    stockAgent.sources.httpSource.port = 8080
    stockAgent.sources.httpSource.handler = org.apache.flume.source.http.JSONHandler
    stockAgent.sources.httpSource.channels = memoryChannel fileChannel

    # Memory Channel for processing
    stockAgent.channels.memoryChannel.type = memory
    stockAgent.channels.memoryChannel.capacity = 10000
    stockAgent.channels.memoryChannel.transactionCapacity = 1000

    # File Channel for durability
    stockAgent.channels.fileChannel.type = file
    stockAgent.channels.fileChannel.capacity = 1000000
    stockAgent.channels.fileChannel.transactionCapacity = 10000
    stockAgent.channels.fileChannel.checkpointDir = /var/flume/checkpoint
    stockAgent.channels.fileChannel.dataDirs = /var/flume/data

    # HDFS Sink
    stockAgent.sinks.hdfsSink.type = hdfs
    stockAgent.sinks.hdfsSink.channel = fileChannel
    stockAgent.sinks.hdfsSink.hdfs.path = /stock_data/%Y/%m/%d
    stockAgent.sinks.hdfsSink.hdfs.fileType = DataStream
    stockAgent.sinks.hdfsSink.hdfs.writeFormat = Text
    stockAgent.sinks.hdfsSink.hdfs.rollInterval = 3600
    stockAgent.sinks.hdfsSink.hdfs.rollSize = 0
    stockAgent.sinks.hdfsSink.hdfs.rollCount = 0
    stockAgent.sinks.hdfsSink.hdfs.filePrefix = stock_data
    """
    
    with open('flume_stock_config.conf', 'w') as f:
        f.write(flume_config)
    
    print("Flume configuration created: flume_stock_config.conf")
    return 'flume_stock_config.conf'

def setup_flume_pipeline():
    """
    Set up the Flume pipeline for data ingestion
    """
    # Create Flume configuration
    flume_config_file = create_flume_config()
    
    # Command to start Flume agent (to be executed in shell)
    flume_start_cmd = f"flume-ng agent --conf /etc/flume --conf-file {flume_config_file} --name stockAgent -Dflume.root.logger=INFO,console"
    
    print(f"To start Flume agent, run: {flume_start_cmd}")
    return flume_start_cmd

###########################################
# 2. STOCK DATA COLLECTION
###########################################

def generate_stock_data_collection_script():
    """
    Generate a Python script to collect stock data from APIs and push to Flume
    """
    collection_script = """
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import os

# List of stock symbols to track
STOCK_SYMBOLS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ']

def collect_historical_data():
    """Collect 10 years of historical data for specified stocks"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)  # 10 years ago
    
    for symbol in STOCK_SYMBOLS:
        print(f"Collecting historical data for {symbol}...")
        stock = yf.Ticker(symbol)
        hist_data = stock.history(start=start_date.strftime('%Y-%m-%d'), 
                                end=end_date.strftime('%Y-%m-%d'),
                                interval='1d')
        
        # Save to CSV for initial ingestion
        output_file = f"stock_data_{symbol}_historical.csv"
        hist_data.to_csv(output_file)
        
        # Push to Flume HTTP source
        send_to_flume(symbol, hist_data)
        
        print(f"Data saved to {output_file}")

def send_to_flume(symbol, data):
    """Send data to Flume HTTP source"""
    FLUME_URL = "http://localhost:8080"
    
    # Convert dataframe to list of records
    records = []
    for date, row in data.iterrows():
        record = {
            'symbol': symbol,
            'date': date.strftime('%Y-%m-%d'),
            'open': float(row['Open']),
            'high': float(row['High']),
            'low': float(row['Low']),
            'close': float(row['Close']),
            'volume': int(row['Volume']),
            'timestamp': int(time.time())
        }
        records.append(record)
    
    # Send in batches to avoid overwhelming the server
    BATCH_SIZE = 100
    for i in range(0, len(records), BATCH_SIZE):
        batch = records[i:i+BATCH_SIZE]
        try:
            response = requests.post(FLUME_URL, json=batch, 
                                    headers={'Content-Type': 'application/json'})
            print(f"Batch {i//BATCH_SIZE + 1} sent to Flume: {response.status_code}")
        except Exception as e:
            print(f"Error sending to Flume: {e}")

def collect_daily_data():
    """Collect daily data for the specified stocks"""
    for symbol in STOCK_SYMBOLS:
        try:
            # Get today's data
            stock = yf.Ticker(symbol)
            today_data = stock.history(period='1d')
            
            # Format data for Flume
            if not today_data.empty:
                record = {
                    'symbol': symbol,
                    'date': datetime.now().strftime('%Y-%m-%d'),
                    'open': float(today_data['Open'].iloc[0]),
                    'high': float(today_data['High'].iloc[0]),
                    'low': float(today_data['Low'].iloc[0]),
                    'close': float(today_data['Close'].iloc[0]),
                    'volume': int(today_data['Volume'].iloc[0]),
                    'timestamp': int(time.time())
                }
                
                # Send to Flume
                response = requests.post("http://localhost:8080", 
                                       json=[record],
                                       headers={'Content-Type': 'application/json'})
                print(f"Daily data for {symbol} sent to Flume: {response.status_code}")
            else:
                print(f"No data available for {symbol} today")
                
        except Exception as e:
            print(f"Error collecting data for {symbol}: {e}")

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1 and sys.argv[1] == "--historical":
        # Collect historical data for initial load
        collect_historical_data()
    else:
        # Default: collect daily data
        collect_daily_data()
    """
    
    with open('stock_data_collector.py', 'w') as f:
        f.write(collection_script)
    
    print("Stock data collection script created: stock_data_collector.py")
    return 'stock_data_collector.py'

###########################################
# 3. SPARK PROCESSING
###########################################

def initialize_spark_session():
    """
    Initialize SparkSession for PySpark processing
    """
    spark = SparkSession.builder \
        .appName("StockPriceForecastingPipeline") \
        .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
        .config("spark.executor.memory", "4g") \
        .config("spark.driver.memory", "2g") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.sql.session.timeZone", "UTC") \
        .getOrCreate()
    
    return spark

def data_cleaning_and_preparation(spark):
    """
    Clean and prepare stock data using PySpark
    """
    # Define schema for stock data
    stock_schema = StructType([
        StructField("symbol", StringType(), True),
        StructField("date", DateType(), True),
        StructField("open", DoubleType(), True),
        StructField("high", DoubleType(), True),
        StructField("low", DoubleType(), True),
        StructField("close", DoubleType(), True),
        StructField("volume", DoubleType(), True),
        StructField("timestamp", DoubleType(), True)
    ])
    
    # Read data from HDFS
    raw_stock_df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "false") \
        .schema(stock_schema) \
        .csv("hdfs://localhost:9000/stock_data/*/*/*/*")
    
    # Register as temp view for SQL operations
    raw_stock_df.createOrReplaceTempView("raw_stock_data")
    
    # Basic cleaning
    cleaned_df = spark.sql("""
        SELECT
            symbol,
            date,
            open,
            high,
            low,
            close,
            volume,
            timestamp
        FROM raw_stock_data
        WHERE
            open IS NOT NULL
            AND close IS NOT NULL
            AND high IS NOT NULL
            AND low IS NOT NULL
            AND volume IS NOT NULL
            AND volume > 0
            AND open > 0
            AND close > 0
            AND high > 0
            AND low > 0
            AND high >= low
    """)
    
    # Create window spec for time-based calculations
    window_spec = Window.partitionBy("symbol").orderBy("date")
    
    # Calculate additional features
    enhanced_df = cleaned_df \
        .withColumn("prev_close", lag("close", 1).over(window_spec)) \
        .withColumn("daily_return", when(col("prev_close").isNotNull(), 
                                      (col("close") - col("prev_close")) / col("prev_close"))
                  .otherwise(lit(0.0))) \
        .withColumn("price_range", col("high") - col("low")) \
        .withColumn("day_of_week", date_format(col("date"), "u").cast("int")) \
        .withColumn("month", date_format(col("date"), "M").cast("int")) \
        .withColumn("year", date_format(col("date"), "yyyy").cast("int"))
    
    # Calculate moving averages
    enhanced_df = enhanced_df \
        .withColumn("ma5", expr("avg(close) OVER (PARTITION BY symbol ORDER BY date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)")) \
        .withColumn("ma20", expr("avg(close) OVER (PARTITION BY symbol ORDER BY date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW)")) \
        .withColumn("ma50", expr("avg(close) OVER (PARTITION BY symbol ORDER BY date ROWS BETWEEN 49 PRECEDING AND CURRENT ROW)")) \
        .withColumn("ma200", expr("avg(close) OVER (PARTITION BY symbol ORDER BY date ROWS BETWEEN 199 PRECEDING AND CURRENT ROW)"))
    
    # Calculate volatility (20-day standard deviation of returns)
    enhanced_df = enhanced_df \
        .withColumn("volatility_20d", expr("stddev(daily_return) OVER (PARTITION BY symbol ORDER BY date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW)"))
    
    # Fill null values for calculated columns
    enhanced_df = enhanced_df \
        .na.fill({
            "prev_close": 0.0,
            "daily_return": 0.0,
            "ma5": 0.0,
            "ma20": 0.0, 
            "ma50": 0.0,
            "ma200": 0.0,
            "volatility_20d": 0.0
        })
    
    # Save processed data back to HDFS
    enhanced_df.write \
        .mode("overwrite") \
        .partitionBy("symbol", "year", "month") \
        .parquet("hdfs://localhost:9000/processed_stock_data/")
    
    print("Data cleaning and preparation completed")
    return enhanced_df

def perform_stock_analysis(spark, stock_df, output_path="hdfs://localhost:9000/stock_analysis_results/"):
    """
    Perform various stock market analyses
    """
    # Register dataframe as temp view
    stock_df.createOrReplaceTempView("stock_data")
    
    # 1. Correlation analysis between stocks
    correlation_df = spark.sql("""
        WITH daily_returns AS (
            SELECT 
                a.date,
                a.symbol as symbol_a,
                b.symbol as symbol_b,
                a.daily_return as return_a,
                b.daily_return as return_b
            FROM stock_data a
            JOIN stock_data b ON a.date = b.date AND a.symbol < b.symbol
        )
        
        SELECT 
            symbol_a,
            symbol_b,
            corr(return_a, return_b) as correlation
        FROM daily_returns
        GROUP BY symbol_a, symbol_b
        ORDER BY correlation DESC
    """)
    
    # 2. Volatility analysis
    volatility_df = spark.sql("""
        SELECT
            symbol,
            year,
            avg(volatility_20d) as avg_volatility,
            max(volatility_20d) as max_volatility,
            min(volatility_20d) as min_volatility
        FROM stock_data
        WHERE volatility_20d IS NOT NULL
        GROUP BY symbol, year
        ORDER BY symbol, year
    """)
    
    # 3. Performance analysis
    performance_df = spark.sql("""
        WITH yearly_prices AS (
            SELECT
                symbol,
                year,
                first_value(close) OVER (PARTITION BY symbol, year ORDER BY date) as start_price,
                last_value(close) OVER (PARTITION BY symbol, year ORDER BY date 
                    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as end_price
            FROM stock_data
        )
        
        SELECT DISTINCT
            symbol,
            year,
            start_price,
            end_price,
            (end_price - start_price) / start_price * 100 as yearly_return_pct
        FROM yearly_prices
        ORDER BY year DESC, yearly_return_pct DESC
    """)
    
    # 4. Technical indicators analysis
    indicators_df = spark.sql("""
        SELECT
            symbol,
            date,
            close,
            ma20,
            ma50,
            ma200,
            CASE 
                WHEN ma20 > ma50 AND lag(ma20, 1) OVER (PARTITION BY symbol ORDER BY date) <= lag(ma50, 1) OVER (PARTITION BY symbol ORDER BY date)
                THEN 'GOLDEN_CROSS'
                WHEN ma20 < ma50 AND lag(ma20, 1) OVER (PARTITION BY symbol ORDER BY date) >= lag(ma50, 1) OVER (PARTITION BY symbol ORDER BY date)
                THEN 'DEATH_CROSS'
                ELSE 'NONE'
            END as ma_signal
        FROM stock_data
        WHERE ma20 IS NOT NULL AND ma50 IS NOT NULL AND ma200 IS NOT NULL
    """)
    
    # Write results to HDFS
    correlation_df.write.mode("overwrite").parquet(output_path + "correlations/")
    volatility_df.write.mode("overwrite").parquet(output_path + "volatility/")
    performance_df.write.mode("overwrite").parquet(output_path + "performance/")
    indicators_df.write.mode("overwrite").parquet(output_path + "technical_indicators/")
    
    print("Stock analysis completed")
    return {
        "correlation": correlation_df,
        "volatility": volatility_df,
        "performance": performance_df,
        "indicators": indicators_df
    }

###########################################
# 4. TIME SERIES FORECASTING
###########################################

def build_multivariate_forecast_model():
    """
    Build multivariate time series forecasting model using Prophet
    """
    # Create a class for the multivariate model implementation
    class MultiStockForecaster:
        """
        Class to implement multivariate time series forecasting for multiple stocks
        """
        def __init__(self, symbols, spark=None):
            self.symbols = symbols
            self.models = {}
            self.metrics = {}
            self.forecasts = {}
            self.spark = spark
            
        def load_data(self):
            """Load data from processed HDFS location"""
            if self.spark is None:
                self.spark = initialize_spark_session()
                
            # Load all processed stock data
            stock_data = self.spark.read.parquet("hdfs://localhost:9000/processed_stock_data/")
            
            # Convert to pandas for Prophet
            self.data_dict = {}
            for symbol in self.symbols:
                symbol_data = stock_data.filter(col("symbol") == symbol).toPandas()
                if not symbol_data.empty:
                    # Format for Prophet (requires 'ds' for date and 'y' for target)
                    prophet_df = symbol_data[['date', 'close']].rename(columns={'date': 'ds', 'close': 'y'})
                    self.data_dict[symbol] = prophet_df
                else:
                    print(f"No data found for symbol {symbol}")
            
            return self.data_dict
        
        def train_models(self, periods=730):  # 730 days = 2 years
            """Train Prophet models for each stock"""
            if not hasattr(self, 'data_dict'):
                self.load_data()
                
            # Train a model for each stock
            for symbol, data in self.data_dict.items():
                print(f"Training model for {symbol}...")
                
                # Configure the Prophet model with parameters for stock forecasting
                model = Prophet(
                    daily_seasonality=False,
                    weekly_seasonality=True,
                    yearly_seasonality=True,
                    seasonality_mode='multiplicative',
                    interval_width=0.95,
                    changepoint_prior_scale=0.05
                )
                
                # Add additional regressors if available in the future
                
                # Train the model
                model.fit(data)
                
                # Store the model
                self.models[symbol] = model
                
                # Create future dataframe for prediction
                future = model.make_future_dataframe(periods=periods)
                
                # Generate forecast
                forecast = model.predict(future)
                self.forecasts[symbol] = forecast
                
                # Evaluate the model using cross-validation
                try:
                    cv_results = cross_validation(
                        model=model,
                        initial='730 days',  # Use 2 years for initial training
                        period='90 days',    # Test on 90-day periods
                        horizon='180 days'   # Forecast 180 days ahead
                    )
                    
                    # Calculate performance metrics
                    metrics = performance_metrics(cv_results)
                    self.metrics[symbol] = metrics
                    
                    print(f"Model training completed for {symbol}")
                    print(f"MAPE: {metrics['mape'].mean():.2f}%")
                    print(f"RMSE: {metrics['rmse'].mean():.2f}")
                except Exception as e:
                    print(f"Error during cross-validation for {symbol}: {e}")
                    
            return self.models
        
        def save_models(self, base_path="/models/"):
            """Save trained models to disk"""
            if not os.path.exists(base_path):
                os.makedirs(base_path)
                
            for symbol, model in self.models.items():
                model_path = os.path.join(base_path, f"{symbol}_model.json")
                with open(model_path, 'w') as f:
                    json.dump(model.to_json(), f)  # Prophet has to_json() method
                    
                # Also save the forecasts
                forecast_path = os.path.join(base_path, f"{symbol}_forecast.csv")
                self.forecasts[symbol].to_csv(forecast_path)
                
            print(f"Models saved to {base_path}")
            return base_path
        
        def load_models(self, base_path="/models/"):
            """Load trained models from disk"""
            self.models = {}
            for symbol in self.symbols:
                model_path = os.path.join(base_path, f"{symbol}_model.json")
                if os.path.exists(model_path):
                    with open(model_path, 'r') as f:
                        model_json = json.load(f)
                        model = Prophet.from_json(model_json)
                        self.models[symbol] = model
                        
                    # Also load forecasts if available
                    forecast_path = os.path.join(base_path, f"{symbol}_forecast.csv") 
                    if os.path.exists(forecast_path):
                        self.forecasts[symbol] = pd.read_csv(forecast_path)
                        
            print(f"Models loaded from {base_path}")
            return self.models
            
        def get_forecast(self, symbol, days=30):
            """Get forecast for a specific symbol"""
            if symbol not in self.forecasts:
                raise ValueError(f"No forecast available for {symbol}")
                
            forecast = self.forecasts[symbol]
            # Get most recent forecasts
            latest_date = forecast['ds'].max()
            start_date = latest_date - timedelta(days=30)  # Include some history
            
            # Filter forecast to include recent history and future predictions
            recent_forecast = forecast[forecast['ds'] >= start_date].tail(days + 30)
            
            return recent_forecast
            
        def get_correlation_forecast(self):
            """Analyze correlations in forecasted values"""
            if not self.forecasts:
                raise ValueError("No forecasts available")
                
            # Combine forecasts from different stocks
            combined_forecast = pd.DataFrame({
                'ds': next(iter(self.forecasts.values()))['ds']  # Get dates from first forecast
            })
            
            # Add forecasted values for each stock
            for symbol, forecast in self.forecasts.items():
                combined_forecast[symbol] = forecast['yhat'].values
                
            # Calculate correlation matrix for future values only
            today = datetime.now().strftime('%Y-%m-%d')
            future_data = combined_forecast[combined_forecast['ds'] > today]
            
            # Drop the date column for correlation calculation
            correlation_matrix = future_data.drop(columns=['ds']).corr()
            
            return correlation_matrix
        
    # Return the forecaster class
    return MultiStockForecaster

###########################################
# 5. MODEL DEPLOYMENT
###########################################

def create_model_deployment():
    """
    Create deployment code for the forecasting model
    """
    deployment_code = """
from flask import Flask, request, jsonify
import pandas as pd
import json
import os
from prophet import Prophet
from datetime import datetime, timedelta

app = Flask(__name__)

# Load models
MODEL_DIR = "/models/"
SYMBOLS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ']
models = {}

def load_models():
    """Load all available models"""
    global models
    for symbol in SYMBOLS:
        model_path = os.path.join(MODEL_DIR, f"{symbol}_model.json")
        if os.path.exists(model_path):
            with open(model_path, 'r') as f:
                model_json = json.load(f)
                models[symbol] = Prophet.from_json(model_json)
                print(f"Loaded model for {symbol}")

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({"status": "healthy", "models_loaded": list(models.keys())})

@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint for stock price prediction
    Expected JSON input:
    {
        "symbol": "AAPL",
        "days": 30
    }
    """
    try:
        data = request.json
        symbol = data.get('symbol')
        days = int(data.get('days', 30))
        
        if symbol not in models:
            return jsonify({"error": f"Model not found for {symbol}"}), 404
        
        # Create future dataframe
        future = models[symbol].make_future_dataframe(periods=days)
        
        # Generate forecast
        forecast = models[symbol].predict(future)
        
        # Format response
        last_30_days = forecast.tail(days)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        formatted_response = []
        
        for _, row in last_30_days.iterrows():
            formatted_response.append({
                'date': row['ds'].strftime('%Y-%m-%d'),
                'predicted_price': float(row['yhat']),
                'lower_bound': float(row['yhat_lower']),
                'upper_bound': float(row['yhat_upper'])
            })
        
        return jsonify({
            'symbol': symbol,
            'predictions': formatted_response
        })
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/predict_all', methods=['GET'])
def predict_all():
    """
    Endpoint to predict prices for all available stocks
    Query parameter: days (default: 7)
    """
    try:
        days = int(request.args.get('days', 7))
        all_predictions = {}
        
        for symbol, model in models.items():
            # Create future dataframe
            future = model.make_future_dataframe(periods=days)
            
            # Generate forecast
            forecast = model.predict(future)
            
            # Format response for this symbol
            last_days = forecast.tail(days)[['ds', 'yhat']]
            symbol_predictions = []
            
            for _, row in last_days.iterrows():
                symbol_predictions.append({
                    'date': row['ds'].strftime('%Y-%m-%d'),
                    'predicted_price': float(row['yhat'])
                })
            
            all_predictions[symbol] = symbol_predictions
        
        return jsonify({
            'predictions': all_predictions
        })
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/update_model', methods=['POST'])
def update_model():
    """
    Endpoint to update a model with new data
    Expected JSON input:
    {
        "symbol": "AAPL",
        "data": [
            {"date": "2025-01-01", "close": 200.0},
            {"date": "2025-01-02", "close": 205.0},
            ...
        ]
    }
    """
    try:
        request_data = request.json
        symbol = request_data.get('symbol')
        new_data = request_data.get('data', [])
        
        if symbol not in models:
            return jsonify({"error": f"Model not found for {symbol}"}), 404
        
        if not new_data:
            return jsonify({"error": "No data provided"}), 400
        
        # Convert input data to Prophet format
        prophet_data = pd.DataFrame([
            {'ds': item['date'], 'y': item['close']} 
            for item in new_data
        ])
        
        # Update the model with new data
        model = models[symbol]
        model.fit(prophet_data)
        
        # Save updated model
        model_path = os.path.join(MODEL_DIR, f"{symbol}_model.json")
        with open(model_path, 'w') as f:
            json.dump(model.to_json(), f)
        
        return jsonify({"status": "success", "message": f"Model for {symbol} updated successfully"})
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Load models when the application starts
load_models()

if __name__ == '__main__':
    print("Starting stock prediction service...")
    app.run(host='0.0.0.0', port=8050)
    """
    
    with open('stock_prediction_service.py', 'w') as f:
        f.write(deployment_code)
    
    print("Model deployment code created: stock_prediction_service.py")
    return 'stock_prediction_service.py'



In [ ]:
###########################################
# 6. VERTEX AI INTEGRATION
###########################################

def create_vertex_ai_deployment_script():
    """
    Create script for deploying the model on Vertex AI
    """
    vertex_deployment_code = """
from google.cloud import aiplatform
from google.cloud.aiplatform import model_monitoring
import os
import json
import argparse

def deploy_to_vertex_ai(
    project_id,
    region,
    model_display_name="stock-price-forecaster",
    model_dir="/models/",
    container_image="gcr.io/{project_id}/stock-price-forecaster:latest"
):
    """Deploy the stock forecasting model to Vertex AI"""
    # Initialize Vertex AI client
    aiplatform.init(project=project_id, location=region)
    
    # Package model artifacts
    model_artifacts_dir = model_dir
    serving_container_image_uri = container_image.format(project_id=project_id)
    
    # Upload model to Vertex AI Model Registry
    model = aiplatform.Model.upload(
        display_name=model_display_name,
        artifact_uri=model_artifacts_dir,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables={
            "MODEL_DIR": "/models"
        }
    )
    
    print(f"Model uploaded to Vertex AI: {model.resource_name}")
    
    # Deploy model to endpoint
    endpoint = model.deploy(
        machine_type="n1-standard-4",
        min_replica_count=1,
        max_replica_count=3,
        accelerator_type=None,
        accelerator_count=0,
        traffic_percentage=100,
        deploy_request_timeout=1800,
        display_name=f"{model_display_name}-endpoint"
    )
    
    print(f"Model deployed to endpoint: {endpoint.resource_name}")
    
    # Set up model monitoring
    monitoring_job = model_monitoring.ModelMonitoringJob.create(
        display_name=f"{model_display_name}-monitoring",
        endpoint=endpoint,
        schedule_hours=24,  # Run every 24 hours
        alerting_config=model_monitoring.AlertingConfig(
            email_alert_config=model_monitoring.EmailAlertConfig(
                user_emails=["admin@example.com"]  # Replace with actual email
            )
        ),
        sampling_rate=0.8,
        analysis_instance_schema_uri=f"gs://{project_id}-model-monitoring/schemas/stock_prediction_schema.yaml",
    )
    
    print(f"Model monitoring job created: {monitoring_job.resource_name}")
    
    return {
        "model": model.resource_name,
        "endpoint": endpoint.resource_name,
        "monitoring_job": monitoring_job.resource_name
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Deploy stock forecasting model to Vertex AI")
    parser.add_argument("--project-id", required=True, help="Google Cloud Project ID")
    parser.add_argument("--region", default="us-central1", help="Google Cloud Region")
    parser.add_argument("--model-name", default="stock-price-forecaster", help="Model display name")
    parser.add_argument("--model-dir", default="/models/", help="Directory containing model artifacts")
    
    args = parser.parse_args()
    
    deploy_to_vertex_ai(
        project_id=args.project_id,
        region=args.region,
        model_display_name=args.model_name,
        model_dir=args.model_dir
    )
"""

    with open('vertex_ai_deployment.py', 'w') as f:
        f.write(vertex_deployment_code)
    
    print("Vertex AI deployment script created: vertex_ai_deployment.py")
    return 'vertex_ai_deployment.py'

def create_docker_files():
    """
    Create Docker files for containerizing the model service
    """
    # Dockerfile
    dockerfile = """
FROM python:3.9-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy model serving code
COPY stock_prediction_service.py .

# Create directory for models
RUN mkdir -p /models

# Copy models (this will be done during the build process)
# COPY ./models/ /models/

# Set environment variables
ENV MODEL_DIR=/models

# Expose port for the prediction service
EXPOSE 8080

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=30s --retries=3 \
  CMD curl -f http://localhost:8080/health || exit 1

# Start the prediction service
CMD ["python", "stock_prediction_service.py"]
"""

    # Requirements file
    requirements = """
prophet==1.1.4
pandas>=1.3.5
numpy>=1.20.0
Flask==2.2.3
gunicorn==20.1.0
"""

    # Build script
    build_script = """
#!/bin/bash
# Script to build and push the Docker image for the stock prediction service

# Configuration
PROJECT_ID=$(gcloud config get-value project)
IMAGE_NAME="stock-price-forecaster"
IMAGE_TAG="latest"
REGION="us-central1"

# Build the Docker image
echo "Building Docker image..."
docker build -t "${IMAGE_NAME}:${IMAGE_TAG}" .

# Tag the image for Google Container Registry
echo "Tagging image for GCR..."
docker tag "${IMAGE_NAME}:${IMAGE_TAG}" "gcr.io/${PROJECT_ID}/${IMAGE_NAME}:${IMAGE_TAG}"

# Push the image to GCR
echo "Pushing image to GCR..."
docker push "gcr.io/${PROJECT_ID}/${IMAGE_NAME}:${IMAGE_TAG}"

echo "Image pushed to gcr.io/${PROJECT_ID}/${IMAGE_NAME}:${IMAGE_TAG}"
"""

    # Write files
    with open('Dockerfile', 'w') as f:
        f.write(dockerfile)
    
    with open('requirements.txt', 'w') as f:
        f.write(requirements)
    
    with open('build_and_push.sh', 'w') as f:
        f.write(build_script)
    
    # Make build script executable
    os.chmod('build_and_push.sh', 0o755)
    
    print("Docker files created: Dockerfile, requirements.txt, build_and_push.sh")
    return ['Dockerfile', 'requirements.txt', 'build_and_push.sh']

###########################################
# 7. AIRFLOW DAG FOR SCHEDULING
###########################################

def create_airflow_dag():
    """
    Create Airflow DAG for scheduling daily data processing
    """
    airflow_dag_code = """
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.bash_operator import BashOperator
from airflow.utils.dates import days_ago
from datetime import timedelta
import os
import sys

# Add project directory to path for imports
sys.path.insert(0, os.path.abspath(os.path.dirname(__file__)))

# Import functions from our pipeline
from stock_data_processor import initialize_spark_session, data_cleaning_and_preparation, perform_stock_analysis
from stock_forecaster import MultiStockForecaster

# Define default arguments
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email': ['your-email@example.com'],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
dag = DAG(
    'stock_price_forecasting_pipeline',
    default_args=default_args,
    description='Pipeline for stock price data processing and forecasting',
    schedule_interval='0 0 * * *',  # Run daily at midnight
    start_date=days_ago(1),
    tags=['stocks', 'forecasting', 'pyspark'],
)

# Define tasks for the DAG
def collect_daily_stock_data():
    """Task to collect daily stock data"""
    import subprocess
    result = subprocess.run(['python', 'stock_data_collector.py'], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        raise Exception(f"Error collecting stock data: {result.stderr}")
    return "Stock data collection completed"

def process_stock_data():
    """Task to process stock data with PySpark"""
    spark = initialize_spark_session()
    cleaned_df = data_cleaning_and_preparation(spark)
    analysis_results = perform_stock_analysis(spark, cleaned_df)
    spark.stop()
    return "Stock data processing completed"

def train_forecast_models():
    """Task to train forecasting models"""
    # Define stock symbols
    symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ']
    
    # Initialize forecaster
    forecaster = MultiStockForecaster(symbols)
    
    # Load data
    forecaster.load_data()
    
    # Train models
    forecaster.train_models(periods=730)  # 2 years
    
    # Save models
    forecaster.save_models("/models/")
    
    return "Forecast models training completed"

def update_prediction_service():
    """Task to update the prediction service with new models"""
    import requests
    try:
        response = requests.get("http://localhost:8050/health")
        print(f"Prediction service health check: {response.json()}")
        
        # No need to manually update as service loads models from /models/ directory
        return "Prediction service updated"
    except Exception as e:
        print(f"Error updating prediction service: {e}")
        return f"Error: {str(e)}"

# Create tasks in the DAG
start_flume_task = BashOperator(
    task_id='start_flume',
    bash_command='flume-ng agent --conf /etc/flume --conf-file flume_stock_config.conf --name stockAgent -Dflume.root.logger=INFO,console',
    dag=dag,
)

collect_data_task = PythonOperator(
    task_id='collect_stock_data',
    python_callable=collect_daily_stock_data,
    dag=dag,
)

process_data_task = PythonOperator(
    task_id='process_stock_data',
    python_callable=process_stock_data,
    dag=dag,
)

train_models_task = PythonOperator(
    task_id='train_forecast_models',
    python_callable=train_forecast_models,
    dag=dag,
)

update_service_task = PythonOperator(
    task_id='update_prediction_service',
    python_callable=update_prediction_service,
    dag=dag,
)

# Define task dependencies
start_flume_task >> collect_data_task >> process_data_task >> train_models_task >> update_service_task
"""

    with open('stock_forecasting_dag.py', 'w') as f:
        f.write(airflow_dag_code)
    
    print("Airflow DAG created: stock_forecasting_dag.py")
    return 'stock_forecasting_dag.py'

###########################################
# 8. MAIN EXECUTION SCRIPT
###########################################

def create_main_script():
    """
    Create main execution script for the pipeline
    """
    main_script = """
#!/usr/bin/env python3
# Stock Price Forecasting Pipeline - Main Execution Script

import os
import argparse
import subprocess
import time
import sys

def setup_environment():
    """Set up the necessary environment for the pipeline"""
    print("Setting up environment...")
    
    # Check if required tools are installed
    required_tools = ['flume-ng', 'spark-submit', 'hadoop']
    missing_tools = []
    
    for tool in required_tools:
        try:
            subprocess.run([tool, '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except FileNotFoundError:
            missing_tools.append(tool)
    
    if missing_tools:
        print(f"ERROR: The following required tools are missing: {', '.join(missing_tools)}")
        print("Please install them before continuing.")
        sys.exit(1)
    
    # Create necessary directories
    dirs_to_create = ['./logs', './models', './data']
    for dir_path in dirs_to_create:
        os.makedirs(dir_path, exist_ok=True)
    
    print("Environment setup completed")

def start_hadoop_flume_services():
    """Start Hadoop and Flume services"""
    print("Starting Hadoop and Flume services...")
    
    # Start Hadoop (if not already running)
    try:
        hadoop_status = subprocess.run(['hadoop', 'dfsadmin', '-report'], 
                                      stdout=subprocess.PIPE, 
                                      stderr=subprocess.PIPE)
        
        if hadoop_status.returncode != 0:
            print("Starting Hadoop services...")
            subprocess.run(['start-dfs.sh'], check=True)
            subprocess.run(['start-yarn.sh'], check=True)
            time.sleep(5)  # Give Hadoop time to start
        else:
            print("Hadoop services already running")
    except Exception as e:
        print(f"Error starting Hadoop: {e}")
        sys.exit(1)
    
    # Configure and start Flume
    print("Setting up Flume configuration...")
    from pipeline import create_flume_config, setup_flume_pipeline
    
    flume_config = create_flume_config()
    flume_cmd = setup_flume_pipeline()
    
    print(f"Start Flume manually with: {flume_cmd}")
    print("Hadoop and Flume services setup completed")

def historical_data_ingestion(args):
    """Ingest historical stock data"""
    print("Starting historical data ingestion...")
    
    # Generate data collection script
    from pipeline import generate_stock_data_collection_script
    collector_script = generate_stock_data_collection_script()
    
    # Execute historical data collection
    try:
        subprocess.run(['python', collector_script, '--historical'], check=True)
        print("Historical data ingestion completed")
    except subprocess.CalledProcessError as e:
        print(f"Error during historical data ingestion: {e}")
        sys.exit(1)

def process_data(args):
    """Process stock data with PySpark"""
    print("Starting data processing with PySpark...")
    
    # Run PySpark processing
    try:
        from pipeline import initialize_spark_session, data_cleaning_and_preparation, perform_stock_analysis
        
        spark = initialize_spark_session()
        cleaned_df = data_cleaning_and_preparation(spark)
        analysis_results = perform_stock_analysis(spark, cleaned_df)
        
        print("Data processing completed")
    except Exception as e:
        print(f"Error during data processing: {e}")
        sys.exit(1)

def train_models(args):
    """Train forecasting models"""
    print("Training forecasting models...")
    
    try:
        from pipeline import build_multivariate_forecast_model
        
        # Define stock symbols
        symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ']
        
        # Initialize forecaster class
        MultiStockForecaster = build_multivariate_forecast_model()
        forecaster = MultiStockForecaster(symbols)
        
        # Load data
        forecaster.load_data()
        
        # Train models
        forecaster.train_models(periods=730)  # 2 years forecast
        
        # Save models
        forecaster.save_models(args.model_dir)
        
        print(f"Models trained and saved to {args.model_dir}")
    except Exception as e:
        print(f"Error during model training: {e}")
        sys.exit(1)

def deploy_service(args):
    """Deploy prediction service"""
    print("Setting up prediction service...")
    
    try:
        # Create deployment code
        from pipeline import create_model_deployment
        service_script = create_model_deployment()
        
        # Create Docker files
        from pipeline import create_docker_files
        docker_files = create_docker_files()
        
        if args.cloud_deploy:
            # Create Vertex AI deployment script
            from pipeline import create_vertex_ai_deployment_script
            vertex_script = create_vertex_ai_deployment_script()
            
            print(f"To deploy to Vertex AI, run: python {vertex_script} --project-id YOUR_PROJECT_ID")
        else:
            # Start local prediction service
            print(f"To start prediction service locally, run: python {service_script}")
        
        print("Deployment setup completed")
    except Exception as e:
        print(f"Error during deployment setup: {e}")
        sys.exit(1)

def setup_scheduling(args):
    """Set up scheduling with Airflow"""
    print("Setting up Airflow DAG for scheduling...")
    
    try:
        from pipeline import create_airflow_dag
        airflow_dag = create_airflow_dag()
        
        airflow_dag_path = os.path.join(args.airflow_dag_path, 'stock_forecasting_dag.py')
        if args.airflow_dag_path != '.':
            # Copy DAG to Airflow DAGs directory
            import shutil
            shutil.copy2(airflow_dag, airflow_dag_path)
        
        print(f"Airflow DAG created at: {airflow_dag_path}")
        print("To enable the DAG in Airflow, make sure the Airflow scheduler is running")
    except Exception as e:
        print(f"Error setting up scheduling: {e}")
        sys.exit(1)

def run_full_pipeline(args):
    """Run the full pipeline"""
    setup_environment()
    start_hadoop_flume_services()
    historical_data_ingestion(args)
    process_data(args)
    train_models(args)
    deploy_service(args)
    setup_scheduling(args)
    print("Full pipeline execution completed")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Stock Price Forecasting Pipeline")
    subparsers = parser.add_subparsers(help="Pipeline commands")
    
    # Setup environment parser
    setup_parser = subparsers.add_parser("setup", help="Set up the environment")
    setup_parser.set_defaults(func=setup_environment)
    
    # Services parser
    services_parser = subparsers.add_parser("services", help="Start Hadoop and Flume services")
    services_parser.set_defaults(func=start_hadoop_flume_services)
    
    # Ingestion parser
    ingestion_parser = subparsers.add_parser("ingest", help="Ingest historical stock data")
    ingestion_parser.set_defaults(func=historical_data_ingestion)
    
    # Processing parser
    processing_parser = subparsers.add_parser("process", help="Process stock data with PySpark")
    processing_parser.set_defaults(func=process_data)
    
    # Model training parser
    training_parser = subparsers.add_parser("train", help="Train forecasting models")
    training_parser.add_argument("--model-dir", default="/models/", help="Directory to save models")
    training_parser.set_defaults(func=train_models)
    
    # Deployment parser
    deploy_parser = subparsers.add_parser("deploy", help="Deploy prediction service")
    deploy_parser.add_argument("--cloud-deploy", action="store_true", help="Deploy to Vertex AI")
    deploy_parser.set_defaults(func=deploy_service)
    
    # Scheduling parser
    schedule_parser = subparsers.add_parser("schedule", help="Set up Airflow scheduling")
    schedule_parser.add_argument("--airflow-dag-path", default=".", help="Path to Airflow DAGs directory")
    schedule_parser.set_defaults(func=setup_scheduling)
    
    # Full pipeline parser
    full_parser = subparsers.add_parser("full", help="Run the full pipeline")
    full_parser.add_argument("--model-dir", default="/models/", help="Directory to save models")
    full_parser.add_argument("--cloud-deploy", action="store_true", help="Deploy to Vertex AI")
    full_parser.add_argument("--airflow-dag-path", default=".", help="Path to Airflow DAGs directory")
    full_parser.set_defaults(func=run_full_pipeline)
    
    # Parse arguments and execute functions
    args = parser.parse_args()
    if hasattr(args, 'func'):
        args.func(args)
    else:
        parser.print_help()
"""

    with open('run_pipeline.py', 'w') as f:
        f.write(main_script)
    
    # Make executable
    os.chmod('run_pipeline.py', 0o755)
    
    print("Main execution script created: run_pipeline.py")
    return 'run_pipeline.py'

###########################################
# 9. PACKAGE THE PIPELINE
###########################################

def create_package_structure():
    """
    Create a package structure for the pipeline
    """
    # Create pipeline module
    pipeline_module = """
# Stock Price Forecasting Pipeline module
# This module contains all the functions used in the pipeline

# Import all functions
from .flume_config import create_flume_config, setup_flume_pipeline
from .data_collection import generate_stock_data_collection_script
from .spark_processing import initialize_spark_session, data_cleaning_and_preparation, perform_stock_analysis
from .forecasting import build_multivariate_forecast_model
from .deployment import create_model_deployment, create_vertex_ai_deployment_script, create_docker_files
from .scheduling import create_airflow_dag

__all__ = [
    'create_flume_config',
    'setup_flume_pipeline',
    'generate_stock_data_collection_script',
    'initialize_spark_session',
    'data_cleaning_and_preparation',
    'perform_stock_analysis',
    'build_multivariate_forecast_model',
    'create_model_deployment',
    'create_vertex_ai_deployment_script',
    'create_docker_files',
    'create_airflow_dag'
]
"""

    # Create directory structure
    os.makedirs('pipeline', exist_ok=True)
    
    # Write init file
    with open('pipeline/__init__.py', 'w') as f:
        f.write(pipeline_module)
    
    # Create module files
    modules = {
        'flume_config.py': """
# Flume configuration module
def create_flume_config():
    \"\"\"
    Create Flume configuration file for stock data ingestion
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass

def setup_flume_pipeline():
    \"\"\"
    Set up the Flume pipeline for data ingestion
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass
""",
        'data_collection.py': """
# Data collection module
def generate_stock_data_collection_script():
    \"\"\"
    Generate a Python script to collect stock data from APIs and push to Flume
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass
""",
        'spark_processing.py': """
# Spark processing module
def initialize_spark_session():
    \"\"\"
    Initialize SparkSession for PySpark processing
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass

def data_cleaning_and_preparation(spark):
    \"\"\"
    Clean and prepare stock data using PySpark
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass

def perform_stock_analysis(spark, stock_df, output_path="hdfs://localhost:9000/stock_analysis_results/"):
    \"\"\"
    Perform various stock market analyses
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass
""",
        'forecasting.py': """
# Forecasting module
def build_multivariate_forecast_model():
    \"\"\"
    Build multivariate time series forecasting model using Prophet
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass
""",
        'deployment.py': """
# Deployment module
def create_model_deployment():
    \"\"\"
    Create deployment code for the forecasting model
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass

def create_vertex_ai_deployment_script():
    \"\"\"
    Create script for deploying the model on Vertex AI
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass

def create_docker_files():
    \"\"\"
    Create Docker files for containerizing the model service
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass
""",
        'scheduling.py': """
# Scheduling module
def create_airflow_dag():
    \"\"\"
    Create Airflow DAG for scheduling daily data processing
    \"\"\"
    # Function implementation goes here
    # This is a placeholder - the actual implementation is in the main script
    pass
"""
    }
    
    # Write module files
    for filename, content in modules.items():
        with open(f'pipeline/{filename}', 'w') as f:
            f.write(content)
    
    # Create setup.py
    setup_py = """
from setuptools import setup, find_packages

setup(
    name="stock_forecasting_pipeline",
    version="0.1.0",
    packages=find_packages(),
    install_requires=[
        "pandas>=1.3.5",
        "numpy>=1.20.0",
        "matplotlib>=3.5.1",
        "pyspark>=3.2.1",
        "prophet>=1.1.1",
        "flask>=2.2.3",
        "google-cloud-aiplatform>=1.16.0",
        "apache-airflow>=2.3.0",
        "yfinance>=0.1.74"
    ],
    author="Your Name",
    author_email="your.email@example.com",
    description="Stock Price Forecasting Pipeline with Flume, HDFS, and PySpark",
    keywords="stock, forecasting, pyspark, flume, hdfs",
    python_requires=">=3.8",
)
"""
    
    with open('setup.py', 'w') as f:
        f.write(setup_py)
    
    # Create README.md
    readme = """
# Stock Price Forecasting Pipeline

A comprehensive pipeline for stock data processing and forecasting using Flume, HDFS, and PySpark on Vertex AI Workbench.

## Features

- Ingests historical stock data (past 10 years) using Flume
- Stores data in HDFS
- Performs data cleaning and analysis with PySpark
- Builds multivariate time series forecasting models
- Deploys models for real-time prediction

## Requirements

- Apache Flume
- Apache Hadoop (HDFS)
- Apache Spark
- Apache Airflow
- Prophet
- Google Cloud Vertex AI

## Installation

```bash
pip install -e .
```

## Usage

```bash
# Run the full pipeline
./run_pipeline.py full

# Or run individual steps
./run_pipeline.py setup
./run_pipeline.py services
./run_pipeline.py ingest
./run_pipeline.py process
./run_pipeline.py train
./run_pipeline.py deploy
./run_pipeline.py schedule
```

## Deployment Options

- Local deployment with Flask
- Cloud deployment on Google Cloud Vertex AI

## License

MIT
"""
    
    with open('README.md', 'w') as f:
        f.write(readme)
    
    print("Package structure created")
    return 'setup.py'

if __name__ == "__main__":
    # Create the pipeline
    print("Creating Stock Price Forecasting Pipeline...")
    
    # Create Flume configuration
    create_flume_config()
    
    # Create data collection script
    generate_stock_data_collection_script()
    
    # Create model deployment code
    create_model_deployment()
    
    # Create Vertex AI deployment script
    create_vertex_ai_deployment_script()
    
    # Create Docker files
    create_docker_files()
    
    # Create Airflow DAG
    create_airflow_dag()
    
    # Create main script
    create_main_script()
    
    # Create package structure
    create_package_structure()
    
    print("\nPipeline creation completed!")
    print("To run the pipeline, execute: ./run_pipeline.py full")